In [1]:
%matplotlib inline

In [2]:
import os
import copy
import glob
import subprocess

import healpy as hp
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt

In [3]:
path = '/Users/jiaxuanli/Research/SideProjects/low-z-galaxy-survey/'
mask = hp.read_map('s18a_fdfc_hp_contarea.fits', nest=True, dtype = np.bool)

NSIDE = 1024
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT


In [4]:
indices_map = np.where(mask)[0]
nside = hp.get_nside(mask)

In [5]:
os.chdir(path + 'sweep/')
filename = glob.glob('*')
os.chdir('..')

In [6]:
%%time
for name in filename:
    data = Table.read('./sweep/' + name)
    phi = np.radians(data['RA'])
    theta = np.radians(90.0 - data['DEC'])
    indices_obj = hp.ang2pix(nside, theta, phi, nest=True)
    sel = np.in1d(indices_obj, indices_map)
    data_use = data[sel]
    if data['TYPE'].dtype.descr[0][1][1:]=='S4':
        psf_mask = (data_use['TYPE']=='PSF ')
        data_use = data_use[~psf_mask]
    if data['TYPE'].dtype.descr[0][1][1:]=='S3':
        psf_mask = (data_use['TYPE']=='PSF')
        data_use = data_use[~psf_mask]
    if len(data_use)!=0:
        data_use.write('./HSCfootprint/HSC-' + name, format='fits')

CPU times: user 4min 33s, sys: 21.5 s, total: 4min 54s
Wall time: 4min 59s


# [TOPCAT](http://www.star.bris.ac.uk/~mbt/topcat/) and [STILTS](http://www.star.bris.ac.uk/~mbt/stilts/) are required. To run these, you need to install Java. You should put `stilts.jar` under the path defined above.

## Match the sweep files with HSC catalog, with precision = 2 arcsec. Then save them to `matched` folder.

In [21]:
os.chdir(path + 'HSCfootprint/')
footprint_filename = glob.glob('*')

os.chdir('..')
fh = open('footprint-filename.txt', 'w')
for name in footprint_filename:
    fh.write(path + 'HSCfootprint/' + name)
    fh.write('\n')
fh.close()

In [25]:
%%time
precision = 3 #arcsec

for name in footprint_filename:
    p = subprocess.Popen('java -jar stilts.jar tmatch2 in1=dr2_s18a_wide_specz.fits in2=./HSCfootprint/' + name + ' out=./matched/match' + str(precision) + '-' + name + ' matcher=sky values1="specz_ra specz_dec" values2="RA DEC" params="' + str(precision) + '"'
                         , shell = True)
    p.wait()

CPU times: user 9.17 ms, sys: 80.9 ms, total: 90 ms
Wall time: 40.7 s


In [27]:
os.chdir(path + 'matched/')
matched_filename = glob.glob('*')

os.chdir('..')
fh = open('matched-filename.txt', 'w')
for name in matched_filename:
    fh.write(path + 'matched/' + name)
    fh.write('\n')
fh.close()

## Find out those galaxies no contained in HSC catalog, and save then into `not-matched` folder.

In [28]:
%%time
precision = 3 #arcsec

for name in footprint_filename:
    p = subprocess.Popen('java -jar stilts.jar tmatch2 join=2not1 in1=dr2_s18a_wide_specz.fits in2=./HSCfootprint/'+name+' out=./not-matched/not-match'+str(precision)+'-'+name+' matcher=sky values1="specz_ra specz_dec" values2="ra dec" params="'+str(precision)+'"', shell=True)
    p.wait()

CPU times: user 9.49 ms, sys: 82 ms, total: 91.5 ms
Wall time: 41.5 s


## Save not-matched files' name to a `.txt` file

In [29]:
# Generate not-matched filenames #
os.chdir(path + 'not-matched/')
not_matched_filename = glob.glob('*')
os.chdir('..')
fh = open('not-matched-filename.txt', 'w')
for name in not_matched_filename:
    fh.write(path + 'not-matched/' + name)
    fh.write('\n')
fh.close()

## Match these `not-matched` files with GAMA catalog

In [30]:
%%time
for name in not_matched_filename:
    p = subprocess.Popen('java -jar stilts.jar tmatch2 in1=SpecObjv27.fits in2=./not-matched/'+name+' out=./GAMA/GAMA-match'+str(precision)+'-'+name+' matcher=sky values1="RA DEC" values2="ra dec" params="'+str(precision)+'"', shell=True)
    p.wait()

CPU times: user 9.27 ms, sys: 84.2 ms, total: 93.4 ms
Wall time: 24 s


In [31]:
# Generate GAMA-matched filenames #
os.chdir(path + 'GAMA/')
GAMA_matched_filename = glob.glob('*')
os.chdir('..')
fh = open('GAMA-matched-filename.txt', 'w')
for name in GAMA_matched_filename:
    fh.write(path + 'GAMA/' + name)
    fh.write('\n')
fh.close()

## Concatenate HSC-matched files

In [32]:
os.chdir(path)
os.getcwd()

'/Users/jiaxuanli/Research/SideProjects/low-z-galaxy-survey'

In [34]:
p = subprocess.Popen('java -jar stilts.jar tcat in=@matched-filename.txt out=HSC-match3-combined.fits', shell=True)

## Concatenate GAMA-matched files

In [35]:
os.chdir(path)
os.getcwd()
p = subprocess.Popen('java -jar stilts.jar tcat in=@GAMA-matched-filename.txt out=GAMA-match3-combined.fits', shell=True)

## Concatenate DECaLS files

In [36]:
os.chdir(path)
os.getcwd()
p = subprocess.Popen('java -jar stilts.jar tcat in=@footprint-filename.txt out=HSCfootprint-combined.fits', shell=True)

## The following processes are convenient using TOPCAT instead of STILS.
- Use `Join->Concatenate Tables` in TOPCAT, and assign `RA -> RA_2`, `DEC -> DEC_2`, then concatenate.
- Show columns of `GAMA-match2-combined.fits`, and unselect items behind 19 (`URL_IMG`), but keep `RA_2` and `DEC_2`.
- Then match the concatenated table with `GAMA-match2-combined.fits`, using `RA -> RA_2, DEC -> DEC_2`, select `Join type = all from 1`. 
- In the matched table, unselect column `RA_2, DEC_2, seperation, seperation_1`.
- Save this table.
- You get it! :)